In this post we will look at the momentum strategy from Andreas F. Clenow's book [Stocks on the Move: Beating the Market with Hedge Fund Momentum Strategy](https://amzn.to/2YzEIvL) and backtest its performance using the survivorship bias-free dataset we created in my [last post](/2019/05/creating-a-survivorship-bias-free-sp-500-dataset-with-python/).

Momentum strategies are almost the opposite of mean-reversion strategies. A typical momentum strategy will buy stocks that have been showing an upward trend in hopes that the trend will continue. The momentum strategy defined in Clenow's books trades based upon the following rules:

 * Trade once a week. In his book, Clenow trades every Wednesday, but as he notes, which day is completely arbitrary.
 
 
 * Rank stocks in the S&P 500 based on momentum. Momentum is calculated by multiplying the annualized exponential regression slope of the past 90 days by the $R^2$ coefficient of the regression calculation.
 
 
 * Position size is calculated using the 20-day [Average True Range](https://www.investopedia.com/terms/a/atr.asp) of each stock, multiplied by 10 basis points of the portfolio value.
 
 
 * Only open new positions if the S&P 500 is above its 200-day moving average.
 
 
 * Every week, look to sell stocks that are not in the top 20% momentum ranking, or have fallen below their 100 day moving average. Buy stocks in the top 20% momentum rankings with remaining cash.
 
 
 * Every other week, rebalance existing positions with updated Average True Range values.

Before we backtest the strategy, let's look into the momentum and position size formulas.

## Momentum

As mentioned above, momentum is calculated by multiplying the annualized exponential regression slope of the past 90 days by the $R^2$ coefficient of the regression calculation. To see this in action, let's look at the highest momentum values measured in our dataset. First we'll need to load in the dataset:

In [77]:
from datetime import datetime 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.ioff()

In [78]:
from glob import glob

In [3]:
tickers = glob("survivorship-free/data/*.csv")
# remove the full path
tickers = [ ticker.split("/")[2].split(".")[0]  for ticker in tickers]
# remove the 'tickers' from the list

tickers.remove('tickers')
# print (tickers)

In [4]:
# tickers = pd.read_csv('survivorship-free/data/tickers.csv', header=None)[1].tolist()

stocks = (
    (pd.concat(
        [pd.read_csv(f"survivorship-free/data/{ticker}.csv", index_col='date', parse_dates=True)[
            'close'
        ].rename(ticker)
        for ticker in tickers],
        axis=1,
        sort=True)
    )
)
stocks = stocks.loc[:,~stocks.columns.duplicated()]

In [5]:
print (stocks.tail())

             CSCO    UAL    TROW    ISRG   PRGO    LUK    TPR    DVN  NU  \
date                                                                       
2018-02-22  42.94  66.70  110.42  419.79  87.07  24.48  49.77  30.41 NaN   
2018-02-23  44.00  67.57  112.30  427.51  88.25  24.97  50.65  31.43 NaN   
2018-02-26  45.36  69.26  114.15  432.00  88.30  25.02  51.16  31.61 NaN   
2018-02-27  45.04  67.86  112.39  430.98  82.00  24.56  50.65  31.86 NaN   
2018-02-28    NaN    NaN     NaN     NaN    NaN    NaN    NaN    NaN NaN   

              MRO  ...    TSS     CRM    PGR     WAT    BWA    LRCX    NWL  \
date               ...                                                       
2018-02-22  15.18  ...  86.80  113.00  56.85  204.47  52.24  188.36  26.71   
2018-02-23  15.54  ...  88.82  114.96  57.71  208.47  52.32  193.10  26.80   
2018-02-26  15.34  ...  88.85  116.65  58.85  208.45  51.91  198.43  27.82   
2018-02-27  15.00  ...  87.99  116.47  58.53  206.11  50.09  193.47  26.96   

Now let's create our momentum measurement function. We can compute the exponential regression of a stock by performing linear regression on the natural log of the stock's daily closes:

In [4]:
from scipy.stats import linregress
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2

Now we can apply a rolling 90 day momentum calculation to all of the stocks in our universe:

In [12]:
momentums = stocks.copy(deep=True)
for ticker in tickers:
    momentums[ticker] = stocks[ticker].rolling(90).apply(momentum, raw=False)

Let's look at the 5 stocks with the best momentum values and plot them along with their regression curve.

## Risk Parity Sizing

Clenow's strategy uses risk parity allocation to calculate the position sizes of each stock. Each stock is assigned a size using the following formula:

{% raw %}
$$Size = {{AccountValue\times RiskFactor} \over {{ATR}_{20}}}$$
{% endraw %}

Where ${ATR}_{20}$ is a stock's [Average True Range](https://www.investopedia.com/terms/a/atr.asp) over the past 20 days. The risk factor, in our case, will be 10 basis points (0.1%). This means that if we assume each stock's ATR remains similar in the future, we can expect each stock to have a daily impact of 0.1% of our portfolio. We are essentially normalizing the weights all of the stocks in our portfolio by risk.

Now that we understand how the strategy works, let's backtest it!

# Backtesting

First we'll code the `Momentum` indicator and our strategy:

In [79]:
import backtrader as bt
from scipy.stats import linregress
import collections

In [80]:

# in order to have a purely declarative indicator
# 1- we build a function that computes it
# 2- use bt.ind.OperationN indicator which must have an attribute func defined, which will get period bars passed as argument 
# and will put the return value into the defined line 



class Momentum(bt.Indicator):
    lines = ('trend',)
    #     With a tuple of tuples parameters retain the order of declaration, 
    #     which can be of importance when enumerating them.
    params = dict(period = 90) # or params = (('period', 90),)
    
    def __init__(self):
        self.addminperiod(self.params.period)
        
    def next(self):
        returns = np.log(self.data.get(size=self.p.period))
        x = np.arange(len(returns))
        slope, _, rvalue, _, _ = linregress(x, returns)
        annualized = (1 + slope) ** 252
        self.lines.trend[0] = annualized * (rvalue ** 2)

class Momentum2(bt.ind.PeriodN):
    lines = ('trend',)
    #     With a tuple of tuples parameters retain the order of declaration, 
    #     which can be of importance when enumerating them.
    params = dict(period = 90) # or params = (('period', 90),)

    def next(self):
        returns = np.log(self.data.get(size=self.p.period))
        x = np.arange(len(returns))
        slope, _, rvalue, _, _ = linregress(x, returns)
        annualized = (1 + slope) ** 252
        self.lines.trend[0] = annualized * (rvalue ** 2)

def momentum_func(prices):
    returns = np.log(prices)
    slope, _ , rvalue, _ , _ = linregress(np.arange(len(r)),r)
    annualized = (1 + slope)**252
    return annualized*(rvalue **2)

class Momentum3(bt.ind.OperationN):
    lines =('trend',)
    params = dict(period=50)
    func = momentum_func
    
 

In [90]:
def dayName(day):
    day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
    return (day_name[day.weekday()]) 

dayName(datetime.today())

'Sunday'

In [112]:
# import datetime


class Strategy(bt.Strategy):
    params = dict(
        rankingPerc = 0.20,
        
        riskFactor = 0.001,
        
        momentum  = Momentum, # parametrize the momentum and its period
        momentum_period = 90,
        
        movav = bt.ind.SMA, # parametrize the moving average and its periods
        idx_period = 200,
        
        stock_period = 100,
        
        volatr = bt.ind.ATR, # parametrize the volatility and its period
        vol_period = 20,  
        
        rebal_weekday = 5, # rebalance 5 is friday
    )
    
    def __init__(self):
#         self.i = 0
        self.timer_counter = 1
        self.inds = collections.defaultdict(dict)
        
        self.index = self.datas[0]
        self.stocks = self.datas[1:]
        
        self.idx_mav = self.p.movav(self.index,period = self.p.idx_period) # bt.indicators.SimpleMovingAverage(self.spy.close,
#                                                             period=200)
        self.index_filter = self.index < self.idx_mav

        for d in self.stocks:
#             self.inds[d] = {}
            self.inds[d]["mom"] = self.p.momentum(d,period=self.p.momentum_period)
            self.inds[d]["mav"] = self.p.movav(d,period=self.p.stock_period)
            self.inds[d]["vol"] = self.p.volatr(d,period=self.p.vol_period)
            self.inds[d]["sma_signal"] = d - self.inds[d]["mav"] #  self.p.volatr(d,period=self.p.vol_period)
        
        self.d_with_len = [] # data with len required to make computations
        
        self.add_timer(
            when = bt.Timer.SESSION_START,
            weekdays=[self.p.rebal_weekday],
            weekcarry=True, # if a day isn't there, execute on the next
            cheat = True,
        )
        
#         self.add_timer(
#             when = bt.Timer.SESSION_START,
#             weekdays=[self.p.rebal_weekday],
#             weekcarry=True, # if a day isn't there, execute on the next
#         )
        
    def prenext(self):
        # call next() even when data is not available for all tickers
        
        self.d_with_len = [d for d in self.stocks if len(d)]
        self.next()
        # but there is no safeguard when entering next
        # all the data is not necessaraly available.
        # the default behavior for next when is called by backtrader is that it waits for all the data to be available
    
    
    def nextstart(self):
        # This is called exactly ONCE, when next is 1st called and defaults to
        # call `next`
        self.d_with_len = self.datas  # all data sets fulfill the guarantees now

        self.next()  
        
    
    def next(self):
        pass
        
#         _, isowk, isowkday = self.datetime.date().isocalendar()
#         txt = '{}, {}, isowk {}, isowkday {}'.format(len(self), self.datetime.datetime(), isowk, isowkday)
#         print(txt)
        
        
        
#         self.rankings = list(filter(lambda d: len(d) , self.stocks))
# #         self.rankings = list(filter(lambda d: len(d) > 100, self.stocks))
#         print ("rankings length: {}".format(len(self.rankings)))
#         self.rankings.sort(key=lambda d: self.inds[d]["mom"][0])
#         self.num_stocks = len(self.rankings)
        
        
#         bars_count = len(self) # counts how many bars passed
#         print ("i: {}, len(bar): {} ".format(self.i,len(self)))
        
#         if bars_count % 5 == 0:
#             self.rebalance_portfolio()
#         if bars_count % 10 == 0:
#             self.rebalance_positions()
            
#         if self.i % 5 == 0:
#             self.rebalance_portfolio()
#         if self.i % 10 == 0:
#             self.rebalance_positions()
#         self.i += 1
        
        


    def notify_timer(self,timer,when,*args,**kwargs):
        
        print ('strategy notify_timer with tid {}, when {}'.
              format(timer.p.tid,when))
        
        self.rankings = list(filter(lambda d: len(d) , self.stocks))
#         self.rankings = list(filter(lambda d: len(d) > 100, self.stocks))
        print ("rankings length: {}".format(len(self.rankings)))
        self.rankings.sort(key=lambda d: self.inds[d]["mom"][0])
        self.num_stocks = len(self.rankings)
        
        print ("self.num_stocks: {}".format(self.num_stocks))
        _, isowk, isowkday = self.datetime.date().isocalendar()

        day_name = dayName(self.datetime.date())

#         day_name = ""
        if self.num_stocks > 0 :
            

            
            txt = 'number of bars passed: {}, current date: {}, isowk: {}, isowkday: {}, day name: {}'.format(len(self), self.datetime.datetime(), isowk, isowkday, day_name)
            
            print (txt)
            
            if self.timer_counter % 2 == 0 :
                print ("rebalance_portfolio...")
                self.rebalance_portfolio()
                print ("rebalance_portfolio... done")
            else:
                print ("rebalance_positions")
                self.rebalance_positions()
                print ("rebalance_positions... done")
        
        self.timer_counter +=1

    def log(self, txt, dt=None):
        if self.p.printout:
            dt = dt or self.data.datetime[0]
            dt = bt.num2date(dt)
            print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = 'BUY COMPLETE, %.2f' % order.executed.price
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = 'SELL COMPLETE, %.2f' % order.executed.price
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log('%s ,' % order.Status[order.status])
            pass  # Simply log

        # Allow new orders
        self.orderid = None

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')
        
    def rebalance_portfolio(self):
        # only look at data that we can have indicators for 
#         self.rankings = list(filter(lambda d: len(d) > 100, self.stocks))
#         self.rankings.sort(key=lambda d: self.inds[d]["mom"][0])
#         num_stocks = len(self.rankings)
        
        # sell stocks based on criteria
        for i, d in enumerate(self.rankings):
            if self.getposition(self.data).size:
                if i > self.num_stocks * self.p.rankingPerc or self.inds[d]["sma_signal"]:
                    self.close(d)
        
        if self.index_filter:
            return
        
        # buy stocks with remaining cash
        for i, d in enumerate(self.rankings[:int(self.num_stocks * self.p.rankingPerc)]):
            cash = self.broker.get_cash()
            value = self.broker.get_value()
            if cash <= 0:
                break
            if not self.getposition(self.data).size:
                size = value * self.p.riskFactor / self.inds[d]["vol"]
                self.buy(d, size=size)
                
        
    def rebalance_positions(self):
#         num_stocks = len(self.rankings)
        
        if self.index_filter:
            return

        # rebalance all stocks
        for i, d in enumerate(self.rankings[:int(self.num_stocks * 0.2)]):
            cash = self.broker.get_cash()
            value = self.broker.get_value()
            if cash <= 0:
                break
            size = value * self.p.riskFactor / self.inds[d]["vol"]
            self.order_target_size(d, size)

As we can see in the code, the strategy looks for stocks it needs to sell every week in the `rebalance_portfolio` method and rebalances all of its positions every other week in the `rebalance_positions` method. Now let's run a backtest!

In [ ]:
cerebro = bt.Cerebro(stdstats=False)
cerebro.broker.set_coc(True)

spy = bt.feeds.YahooFinanceData(dataname='SPY',
                                 fromdate=datetime(2012,2,28),
                                 todate=datetime(2018,2,28),
                                 plot=False)

cerebro.adddata(spy)  # add S&P 500 Index

print ("loading the data ...")
for ticker in tickers:
    df = pd.read_csv(f"survivorship-free/data/{ticker}.csv",
                     parse_dates=True,
                     index_col=0)
    if len(df) > 100: # data must be long enough to compute 100 day SMA
        cerebro.adddata(bt.feeds.PandasData(dataname=df, plot=False))
    else:
        print ("ticker: {}, length: {}".format(ticker,len(df)) )
        
print ("loading the data ... done")

cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addstrategy(Strategy)
results = cerebro.run()

loading the data ...
ticker: TPR, length: 80
ticker: NU, length: 47
ticker: CRC, length: 46
ticker: BLD, length: 23
ticker: IQV, length: 60
ticker: Q, length: 52
ticker: CC, length: 23
ticker: WPG, length: 22
ticker: APTV, length: 40
ticker: FHN, length: 84
ticker: DF, length: 64
ticker: HII, length: 19
ticker: COP, length: 42
ticker: CMCSK, length: 52
ticker: CWGL, length: 21
ticker: BTI, length: 24
ticker: SYF, length: 22
ticker: NCLH, length: 80
ticker: X, length: 43
ticker: AA, length: 20
loading the data ... done


In [73]:
cerebro.plot(iplot=False)[0][0]
print(f"Sharpe: {results[0].analyzers.sharperatio.get_analysis()['sharperatio']:.3f}")
print(f"Norm. Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%")
print(f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")

Sharpe: 0.833
Norm. Annual Return: 9.11%
Max Drawdown: 20.55%


In [11]:
cerebro.plot(iplot=False)[0][0]
print(f"Sharpe: {results[0].analyzers.sharperatio.get_analysis()['sharperatio']:.3f}")
print(f"Norm. Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%")
print(f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")

Sharpe: 1.269
Norm. Annual Return: 8.99%
Max Drawdown: 11.71%


As we can see the algorithm performs pretty well. It makes an average of almost 9% a year with a max drawdown of only 11%. Although the S&P 500 slightly outperforms the algorithm over this time period (CAGR of 12%), it does so with more volatility (Max Drawdown of 13.5%, Sharpe of 1.07). Overall, this algorithm provides a good base for a momentum strategy and can likely be improved by altering parameters, applying filters, and adding leverage. I would highly recommend reading Clenow's book [Stocks on the Move: Beating the Market with Hedge Fund Momentum Strategy](https://amzn.to/2YzEIvL), as it provides a much more in depth description as to how the algorithm works, as well as detailed analysis of how it has performed historically.

If you would like to try the the strategy for yourself, you can find [this notebook](https://github.com/teddykoker/blog/tree/master/notebooks) on my Github, along with my [survivorship bias-free dataset](https://github.com/teddykoker/quant/tree/master/survivorship-free)!